# Rossmann销售预测

## 问题描述

Rossmann在全欧洲有超过6000家药店，预测销售额一直是他们商店经理的工作，他们根据直觉来预测，
准确率有很大变化，现在我们要帮助构建一个销售额预测模型，针对位于德国的1115家店进行6周的销
售额预测，对于销售额的预测可以帮助经理们更合理的安排员工上班时间表、销售活动等。

## 问题链接

https://www.kaggle.com/c/rossmann-store-sales

## 基本流程

1. 开发环境初始化。
2. 加载数据。
3. 拼接数据。
4. 数据预处理。
5. 数据挖掘。
6. 特征工程。
7. 模型构建、训练、调参、融合到一个无法提升的地步。
8. 完善各步骤。
9. 生成kaggle的提交文件。
10. 记录kaggle得分情况。
11. 收获。
12. 引用。

## 环境初始化

In [ ]:
import time
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

## 准备数据

相关数据都存放于当前目录的/data/all中

In [ ]:
os.listdir('../input')

### 个别字段含义

    1. Id:测试集内(商店、日期)的组合。
    2. Store:表示每个商店的唯一Id。
    3. Sales:任意一天的销售额，也是我们要预测的字段。
    4. Open:是否开门，0=关门，1=开门。
    5. StateHoliday:国家假日，一般假日国家假期都会关门，所有学校在公共假日都会关门，a=公共假日，b=东部假日，c=圣诞节，0=不是假日。
    6. StoreType:商店类型，有四种，abcd。
    7. Assortment:分类级别，a=基础，b=额外，c=扩展。
    8. CompetitionDistance:竞争对手距离。
    9. CompetitionOpenSince\[Month/Year\]:给出最近竞争对手的开张时间。
    10. Promo:表示商店当天是否进行促销？
    11. Promo2:表示商店是否进行持续的促销活动，0=没有参数，1=参与。
    12. Promo2Since\[Year/Week\]:商店开始持续促销的年/星期。
    13. PromoInterval:持续促销活动开始的间隔，"Feb,May,Aug,Nov"表示给定商店某一年的2589月开始持续促销活动。

### 读取数据

In [ ]:
base_path = '../input/'

In [ ]:
train_data = pd.read_csv(base_path+'train.csv')
test_data = pd.read_csv(base_path+'test.csv')
store_data = pd.read_csv(base_path+'store.csv')

### 字段数据类型转换

In [ ]:
train_data.Date = pd.to_datetime(train_data.Date)
test_data.Date = pd.to_datetime(test_data.Date)
store_data['PromoInterval'] = store_data['PromoInterval'].astype(str)

### 浏览下数据

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
store_data.head(5)

结论：

1. 可以看到train、test和store之间有一个Store字段是相同的，可以同于链接两张表。
2. train中的Store、DayOfWeek、Date、Open、Promo、StateHolidy、SchoolHoliday都不能直接使用，因为不是数值型。
3. store中除COmpetitionDistance外均不能直接使用。

能够看到有大量字段属于枚举型、时间序列，这些字段都要经过处理，否则会影响预测结果。

### 数据的统计值

In [ ]:
train_data.info()

可以看到，总共有1017209条数据，数据量不算很大，各字段都是完整的，无null字段，这是个好消息，对于train部分不需要做异常数据处理了。

In [ ]:
test_data.info()

Open字段有11个NaN，这个直接用1，表示开门来填充即可。

In [ ]:
store_data.info()

对于Store数据来说，总共有1115条数据，对应分布于德国的1115间商店，而其中CompetitionOpenSinceMonth、CompetitionOpenSinceYear只有761条数据，也就是说有354个商店没有对应的竞争对手的开张日期字段，而Promo2SinceWeek、Promo2SinceYear、PromoInterval只有571条，也就是说有将近一半的商店是没有持续促销活动的，而还有3个商店没有竞争对手的距离。

In [ ]:
store_data[store_data['CompetitionDistance'].isnull()]

可以看到有三家店是没有CompetitionDistance，不知道是没有竞争对手还是什么情况，不过既然只有3条，那么直接用平均值填充好了，本身想删掉的，但是考虑到store的信息要链接到train中，这里的3条数据对应train中可就是3\*N条了，因此不删了。

In [ ]:
store_data[store_data['CompetitionOpenSinceMonth'].isnull()][:5]

看到对于CompetitionOpenSinceMonth、CompetitionOpenSinceYear的数据，并不是没有竞争对手，只是缺失了对手的开张日期而已，同时缺失了大概1/3的数据，不算很多，那么我们在后面将他们补全吧，补全方式使用其他数据该字段的平均值。

In [ ]:
store_data[store_data['Promo2SinceWeek'].isnull()][:5]

而对于Promo2SinceWeek、Promo2SinceYear、PromoInterval来说，它们并不是缺失，而是当Promo2为0，即没有持续的促销活动时，这三个字段都是NaN，这个是正常的，后面考虑下如何处理Promo2系的字段吧，毕竟缺失较多，或者可以将数据按照是否有Promo2划分开两部分，分别进行训练和预测，恩恩，感觉可以试试看。

## 数据预处理

### Open

In [ ]:
test_data.Open.fillna(1, inplace=True)

### CompetitionOpenSinceMonth、CompetitionOpenSinceYear填充

使用该字段不为NaN的字段的数据的median填充，注意要取整。

In [ ]:
store_data.CompetitionOpenSinceYear.fillna(store_data.CompetitionOpenSinceYear.median(), inplace=True)
store_data.CompetitionOpenSinceMonth.fillna(store_data.CompetitionOpenSinceMonth.median(), inplace=True)

### CompetitionDistance填充

直接使用该字段不为NaN的median填充。

In [ ]:
store_data.CompetitionDistance.fillna(store_data.CompetitionDistance.median(), inplace=True)

## 数据拼接

### Train和Store的链接

根据Store字段将二者链接起来。

In [ ]:
train_all = pd.merge(train_data, store_data)
train_all.head(5)

In [ ]:
train_all.info()

### Test和Store的链接

同样也将Test和Store链接，方便后续对test进行预测时使用，后续对链接后的处理，同理都要应用到Test+Store的链接数据上。

In [ ]:
test_all = pd.merge(test_data, store_data)

In [ ]:
test_all.info()

### 促销信息缺失

In [ ]:
train_all[train_all['Promo2']==0][:3]

可以看到这部分信息缺失是因为没有参与促销活动。

## 将数据按照日期排序

重要：由于后面需要将最近6周的数据抽取出来作为验证集数据，因此如果训练数据没有通过日期排序的话，会出现整个验证集数据都是Store为1的数据，那么我们根据其他商店的数据去预测Store为1的商店的预测，可想而知，肯定会出现验证集上早早的就无法降低RMSPE值的问题，因为训练的数据本身就不合理。

In [ ]:
train_all = train_all.sort_values(['Date'],ascending = False)

In [ ]:
train_all[:10] # 排序成功的话，这十条数据应该都是2015年7月31号的不同商店的数据才对吧，妈呀，坑死人了。。。。。。

## 数据挖掘：日期信息、促销信息、竞争对手信息

### 日期信息：提取年、月、日、WeekOfYear（对应Promo2SinceWeek）、是否工作日

对于时间字段，最早我的做法是将其当做枚举来处理，这么做有两个问题：
1. 枚举量很大，影响后续PCA等的效果，实际上如果特征数量有限就不需要PCA了，毕竟PCA算是牺牲了一部分数据表现。
2. 训练数据和测试数据在枚举变量One-Hot后不能对齐。
3. 没有真正对日期信息进行深度的挖掘。

最重要的是时间序列字段对于预测的影响应该是很大的，因为商店的销售应该是具有季节性、日期特殊性的，比如节假日、季节等，因此，通过已有信息挖掘这部分信息出来作为新特征，然后抛弃原有的较原始的信息对于我们的模型来说帮助更大。

提取字段：
1. Year。
2. Quarter。
3. Month。
4. Day。
5. WeekOfYear。
6. IsWorkDay。

In [ ]:
def get_datetime_info(data):
    '''
    data:dataFrame
    return year,quarter,month,day,weekOfYear,isWorkDay
    '''
    return (data.Date.apply(lambda date:date.year), 
            data.Date.apply(lambda date:date.quarter), 
            data.Date.apply(lambda date:date.month), 
            data.Date.apply(lambda date:date.day), 
            data.Date.apply(lambda date:date.weekofyear), 
            data.DayOfWeek.apply(lambda dow:dow<=6)) # 周日不上班

In [ ]:
get_datetime_info(train_all[:1])

#### 可视化分析时间字段相对于销售额的影响

In [ ]:
train_data.Date.min()

In [ ]:
train_data.Date.max()

In [ ]:
train_data.Date.unique()[:10]

日期是连续的，这有助于我们计算一些统计信息。

In [ ]:
print('国家放假平均销售额：'+str(train_data[train_data['SchoolHoliday'] == 1].Sales.mean()))
print('国家不放假平均销售额：'+str(train_data[train_data['SchoolHoliday'] == 0].Sales.mean()))

In [ ]:
print('国家放假平均销售额：'+str(train_data[train_data['StateHoliday'] != 0].Sales.mean()))
print('国家不放假平均销售额：'+str(train_data[train_data['StateHoliday'] == 0].Sales.mean()))

可以看到仅仅是是否放假，对销售额的影响不大，下面我们看看放假期间的销售额变化。

In [ ]:
X=[]
Y=[]
X_=[]
Y_=[]
train_data_store1 = train_all[train_all.Store==1][::-1].reset_index()
train_data_store1 = train_data_store1[:360]
holiday_start = False
holiday_end = False
x_temp = []
y_temp = []
plt.figure(figsize=(15,15))
for i in range(len(train_data_store1)):
    d = train_data_store1.loc[i]
    if d.SchoolHoliday==1 and (i==0 or train_data_store1.loc[i-1].SchoolHoliday==0):
        X.append(d.Date)
        Y.append(9000)
        holiday_start = True
        holiday_end = False
        x_temp = []
        y_temp = []
    if d.SchoolHoliday==0 and (i==len(train_data_store1)-1 or train_data_store1.loc[i-1].SchoolHoliday==1):
        X.append(d.Date)
        Y.append(9500)
        holiday_end = True
        holiday_start = False
    if holiday_start and not holiday_end:
        x_temp.append(d.Date)
        y_temp.append(d.Sales)
    if holiday_end and not holiday_start:
        X_.append(x_temp)
        Y_.append(y_temp)
        holiday_end = False

for i in range(len(X_)):
    plt.plot(X_[i], Y_[i])
for i in range(0, min(len(X), len(Y)), 2):
    plt.plot(X[i:i+2], Y[i:i+2])
        

假期的销售额幅度不明显，看来假期对销售额的影响应该不大，至少不是主要的因素。

In [ ]:
plt.figure(figsize=(15,15))
plt.plot(train_all[train_all.Store==1].Date[:365], train_all[train_all.Store==1].Sales[:365])
plt.plot(train_all[train_all.Store==1].Date[365:365+365], train_all[train_all.Store==1].Sales[365:365+365])

上述图很重要，分别表示了相邻的两年的销售额走势图，我们可以发现，大体上是一致的，这说明一个问题，销售额跟日期息息相关的，也就是说每一年的同一段时期内，可能销售额都会很接近，这就有点像很多行业有他的季节性特点的感觉，这一信息说明具体的日期对预测是至关重要的，因此我们需要提取出具体的年、月、日信息。

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(train_data[train_data.Store==1].DayOfWeek[365:365+365], train_data[train_data.Store==1].Sales[365:365+365])

In [ ]:
print('1~5平均销售额：'+str(train_data[train_data['DayOfWeek'] <=5].Sales.mean()))
print('6平均销售额：'+str(train_data[train_data['DayOfWeek'] ==6].Sales.mean()))
print('7平均销售额：'+str(train_data[train_data['DayOfWeek'] ==7].Sales.mean()))

可以看到相比于周六，周日对销售额的影响是巨大的，主要是因为很多商店这一天都关门吧。

分析一下每一天跟前一个星期、月、季度、半年、一年时间的平均销售额的对比。

In [ ]:
def get_week_month_season_halfyear_year(train_data_store):
    every_day = train_data_store1.Sales
    last_weeks = []
    last_months = []
    last_seasons = []
    last_halfyears = []
    last_years = []
    for i in range(len(every_day)):
        # week
        sales=0
        count=1
        for j in range(i, i-7 if i-7>0 else 0, -1):
            sales+=every_day[j]
            count+=1.
        last_weeks.append(sales/count)
        # month
        sales=0
        count=1
        for j in range(i, i-30 if i-30>0 else 0, -1):
            sales+=every_day[j]
            count+=1.
        last_months.append(sales/count)
        # season
        sales=0
        count=1
        for j in range(i, i-90 if i-90>0 else 0, -1):
            sales+=every_day[j]
            count+=1.
        last_seasons.append(sales/count)
        # halfyear
        sales=0
        count=1
        for j in range(i, i-180 if i-180>0 else 0, -1):
            sales+=every_day[j]
            count+=1.
        last_halfyears.append(sales/count)
        # year
        sales=0
        count=1
        for j in range(i, i-360 if i-360>0 else 0, -1):
            sales+=every_day[j]
            count+=1.
        last_years.append(sales/count)
    return every_day, last_weeks, last_months, last_seasons, last_halfyears, last_years

train_data_store1 = train_all[train_all['Store']==1][::-1].reset_index()
every_day, last_weeks, last_months, last_seasons, last_halfyears, last_years = get_week_month_season_halfyear_year(train_data_store1)
plt.figure(figsize=(15,15))
plt.plot(every_day, label='every day')
plt.plot(last_weeks, label='week')
plt.plot(last_months, label='month')
plt.plot(last_seasons, label='season')
plt.plot(last_halfyears, label='halfyear')
plt.plot(last_years, label='year')
plt.legend()
plt.show()

可以看到趋势方面最接近的应该是week，下面用相关系数确认以下。

In [ ]:
pd.DataFrame([last_weeks, last_months, last_seasons, last_halfyears, last_years, 
              list(train_data_store1.Customers)]).corrwith(pd.Series(every_day), axis=1)

可以看到，就相关性来看，最高的有0.26，也不算很高，暂时不考虑。

### 持续促销活动相关信息挖掘：PromoIntervel、Promo2SinceWeek、Promo2SinceYear

可以开发两个字段：
1. IsInPromo:分别表示当前店铺某天是否处于持续的促销活动中：根据Date、Promo2以及PromoIntervel。
2. PromoDays:促销活动已经持续的时间：根据Date以及Promo2SinceYear、Promo2SinceWeek。

In [ ]:
def is_in_promo(data):
    '''
    data:DataFrame。
    return:返回bool值的Seris表示当前是否处于活动中。
    '''
    months_str = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    return data.apply(lambda d:False if d.Promo2==0 else (months_str[int(d.Month-1)] in d.PromoInterval), axis=1)

In [ ]:
def get_promo_days(data):
    '''
    return:返回活动已经持续的天数的Series。
    '''
    return data.apply(lambda d:0 if not d.IsInPromo else d.Day, axis=1)

### 竞争对手信息挖掘：CompetitionOpenSinceYear、CompetitionOpenSinceMonth

按照一般的理解上，竞争对手也应该对店铺的销售有很大影响，毕竟同一地区的市场份额是固定的，有一个距离比较近、开张比较久（老字号？）的竞争对手对销售额还是有压力的吧，分析以下。

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(train_all[train_all['CompetitionDistance']<10000].CompetitionDistance, train_all[train_all['CompetitionDistance']<10000].Sales)

可以看到一个奇怪的现象，似乎与我们的预计不符，并不是竞争对手越远，销售额越高，反而有点相反的意思，这个可能是类似商业圈的特点导致的吧，比如这一代有好几家同样的店，那么大家买这类商品时是不是都倾向于去这些地方买呢，那么就有一种互相促进的感觉，这一点是很有意思的点。

下面再看看跟竞争对手开张时间的关系。

增加字段：
1. CompetitionOpenMonths。

In [ ]:
def get_competition_openmonths(data):
    '''
    return:返回截止当前竞争对手的开张时间，月为单位。
    '''
    return data.apply(lambda d:(d.Year-d.CompetitionOpenSinceYear)*12+(d.Month-d.CompetitionOpenSinceMonth), axis=1)

In [ ]:
months = train_all.apply(lambda data:(data.Date.year - data.CompetitionOpenSinceYear)*12+(data.Date.month - data.CompetitionOpenSinceMonth), axis=1)
plt.figure(figsize=(15,15))
plt.scatter(months, train_all.Sales)

可以看到，这张图很明显的看到，当对手开张时间比较短时，店铺的销售额比较大，也就是在同一个区域更加受客户欢迎，这个是跟我们的认知一致的信息。

## 特征工程

### 挖掘到的新字段添加

#### Year、Quarter、Month、Day、WeekOfYear、IsWorkDay

In [ ]:
train_all['Year'], train_all['Quarter'], train_all['Month'], train_all['Day'], train_all['WeekOfYear'], train_all['IsWorkDay'] = get_datetime_info(train_all)

In [ ]:
test_all['Year'], test_all['Quarter'], test_all['Month'], test_all['Day'], test_all['WeekOfYear'], test_all['IsWorkDay'] = get_datetime_info(test_all)

Tips:如果使用train_all.Year增加列，而不是train_all\['Year'\]，后续虽然这个Year可以点出来使用，但是在columns列列表中不存在会有一些问题哦！！！

#### 可视化

In [ ]:
plt.figure(figsize=(15,15))
train_all.groupby(['Quarter']).Sales.mean().plot()

In [ ]:
plt.figure(figsize=(15,15))
train_all.groupby(['Month']).Sales.mean().plot()

In [ ]:
plt.figure(figsize=(15,15))
train_all.groupby(['WeekOfYear']).Sales.mean().plot()

#### IsInPromo、PromoDays

In [ ]:
train_all['IsInPromo'] = is_in_promo(train_all)
train_all.IsInPromo.unique()

In [ ]:
test_all['IsInPromo'] = is_in_promo(test_all)

In [ ]:
train_all['PromoDays'] = get_promo_days(train_all)
train_all.PromoDays.unique()

In [ ]:
test_all['PromoDays'] = get_promo_days(test_all)

#### 可视化

In [ ]:
plt.figure(figsize=(15,15))
train_all.groupby(['IsInPromo']).Sales.mean().plot()

In [ ]:
plt.figure(figsize=(15,15))
train_all.groupby(['PromoDays']).Sales.mean().plot()

上述表示，促销对销售额的影响是非常明显的，但是持续时间同样存在着影响。

#### CompetitionOpenMonths

In [ ]:
train_all['CompetitionOpenMonths'] = get_competition_openmonths(train_all)
train_all.CompetitionOpenMonths.unique()[:10]

In [ ]:
test_all['CompetitionOpenMonths'] = get_competition_openmonths(test_all)

### 无用字段丢弃，注意test.csv是没有Customers数据的哈

Date、Promo2SinceWeek、Promo2SinceYear、PromoInterval、CompetitionOpenSinceMonth、CompetitionOpenSinceYear、Store

In [ ]:
drop_cols = ['Date', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 
             'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Store']
train_all.drop(drop_cols+['Customers'], axis=1, inplace=True)
test_all.drop(drop_cols, axis=1, inplace=True)

### 当前字段

In [ ]:
train_all.columns

In [ ]:
test_all.columns

### 枚举字段One-Hot编码

当前的枚举字段有：
1. StateHoliday:国家假日，一般假日国家假期都会关门，所有学校在公共假日都会关门，a=公共假日，b=东部假日，c=圣诞节，0=不是假日。
2. StoreType:商店类型，有四种，abcd。
3. Assortment:分类级别，a=基础，b=额外，c=扩展。

在模型中应该是类似节点一样的存在，即作为四个节点将数据导向四个方向，因此将a、b、c、d都映射到1,2,3,4上，不使用OneHot编码，毕竟编码后的维度会增加。

In [ ]:
# train_all.head(3)
# train_all = pd.get_dummies(train_all, columns=['StateHoliday', 'StoreType', 'Assortment'])
# train_all.info()
# test_all = pd.get_dummies(test_all, columns=['StateHoliday', 'StoreType', 'Assortment'])
# test_all.info()

In [ ]:
code_map = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, '0':0, 
           1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 0:0}
train_all.StateHoliday = train_all.StateHoliday.map(code_map)
train_all.StoreType = train_all.StoreType.map(code_map)
train_all.Assortment = train_all.Assortment.map(code_map)

test_all.StateHoliday = test_all.StateHoliday.map(code_map)
test_all.StoreType = test_all.StoreType.map(code_map)
test_all.Assortment = test_all.Assortment.map(code_map)

print(train_all.StateHoliday.unique())
print(test_all.Assortment.unique())

## 数值型数据归一化处理

这主要是避免由数值大小导致的字段在对预测结果的影响中权重不一致，因此做归一化处理，即认为每个字段的影响都是一样的。

数值型字段有：
1. Sales：预测值不用管。
2. CompetitionDistance。
3. PromoDays。
4. CompetitionOpenMonths。

但是有个问题，如果对训练数据、测试数据分别进行了归一化，因为归一化使用的min、max不同，是否对预测有影响不可而知，暂时不进行归一化处理。

### 做归一化之前，我们要先查看下数据中是否有异常值，比如极大极小值等，避免对归一化结果造成影响

In [ ]:
#plt.figure(figsize=(15,15))
#plt.plot(train_all.CompetitionDistance)

In [ ]:
print('min:'+str(train_all.CompetitionDistance.min()))

In [ ]:
print('max:'+str(train_all.CompetitionDistance.max()))

恩恩，也没啥问题，不过这个min为20，这个不会是楼上楼下的关系吧，真的不会打起来么。。。。

### 进行归一化

归一化就使用简单的min-max归一化（是否需要看下数值的分布再决定呢，不是有个归一化算法可以减少值间距导致的差异么），因为现在所有字段都可以理解为数值型，因此直接应用到整个DataFrame上也是可以的，不过为了速度就不这么做了。

In [ ]:
# train_all.CompetitionDistance = ((train_all.CompetitionDistance - train_all.CompetitionDistance.min())/(train_all.CompetitionDistance.max() - train_all.CompetitionDistance.min()))
# test_all.CompetitionDistance = ((test_all.CompetitionDistance - test_all.CompetitionDistance.min())/(test_all.CompetitionDistance.max() - test_all.CompetitionDistance.min()))
# print 'min:'+str(train_all.CompetitionDistance.min())
# print 'max:'+str(train_all.CompetitionDistance.max())
# print 'mean:'+str(train_all.CompetitionDistance.mean())

## 将目标字段提取出来

In [ ]:
target_all = train_all.Sales
target_all.head(5)

In [ ]:
train_all = train_all.drop('Sales', axis=1)
print('Sales' in train_all.columns)

可以看到，Sales已经不在训练数据中了。

## 数据集划分

避免每一次模型验证都要上传到kaggle，因此将训练数据划分为训练集和验证集，比例9:1，这样方便自己调试模型，因为最终测试集上是预测6周的数据，因此我们也拿最近的6周出来做验证集数据，由于时间序列是连续的，而销售额又是跟日期强相关的，因此不适用随机划分。

In [ ]:
#from sklearn.cross_validation import train_test_split

#x_train, x_valid, y_train, y_valid = train_test_split(train_all, target_all, test_size=0.1)

x_valid = train_all[:1115*6*7]
x_train = train_all[1115*6*7:]
y_valid = target_all[:1115*6*7]
y_train = target_all[1115*6*7:]

# y做对数处理，数据分布转换
y_train = np.log1p(y_train)
y_valid = np.log1p(y_valid)

In [ ]:
x_train[:5]

In [ ]:
x_valid[:5]

## 基准模型

基准模型采用恒定猜测为mean值的方式。

先将训练集中Sales的mean值计算出来。

In [ ]:
pred_base = np.expm1(y_train).mean()
print('基准模型预测值：'+str(pred_base))

## 性能指标

由于是要提交到kaggle，因此我们选择和kaggle一致的性能指标，即RMSPE。

In [ ]:
def rmspe(y_pred, y_real):
    y_pred = list(y_pred)
    y_real = list(y_real)
    for i in range(len(y_real)):
        if y_real[i]==0:
            y_real[i], y_pred[i] = 1., 1.
    return np.sqrt(np.mean((np.divide(np.subtract(y_real, y_pred),y_real))**2))

跟RMSE不同的是引入了/y_i的处理，这样就不会忽略销量很低的情况了。

## 计算基准模型的性能

In [ ]:
print('基准模型的RMSPE：'+str(rmspe([pred_base]*len(y_valid), np.expm1(y_valid))))

可以看到是一个很高的误差率了，就是不知道跟我们的模型比较如何。

## 主流程

接下来就是我们的模型相关的流程，之前的数据处理、基准模型构建、基准阈值计算、性能指标函数构建已经完成了，后续主要就是PCA以及模型构建、比较、调试的迭代过程了。

模型选择：本来是选择Adaboost，看中它在小数据集上的表现，但是开题报告的审阅导师推荐了xGBoost，去了解了一下，发现如何区别：
1. adaboost：在优化弱分类器时，依赖的是权重的设置，即加大分类错误的数据的权重，而减小分类正确的数据的权重，使得后续的分类器更关注之前分类错误的点。
2. GBDT：相比较adaboost，区别在于它是通过算梯度来定位模型的不足，因此相比较AdaBoost，它能使用更多的目标函数，比如我们的性能指标函数RMSPE。

而XGBoost是在GBDT的基础上的全方位加强版，具体表现在支持线性分类器、加入了正则项控制模型复杂度、学习速率、速度更快，支持并行等，因此我们的模型优先选择**XGBoost**。

### PCA

进行PCA的目的：
1. 去除数据噪音。
2. 降维，我们的维度高达1000+，以当前数据是不可能描述这么多维度的。
3. 更加了解数据，很多字段我们根本不知道跟目标的关系是什么，pca可以帮助我们梳理这一关系。

第一版的时候由于维度非常多，因此我采用了PCA，主要用于降维，以及特征提取，第二版由于去掉了大部分OneHot部分，因此维度并不是很高，先去掉PCA，毕竟PCA对信息是有损失的。

    from sklearn.decomposition import PCA

    pca = PCA(n_components=10).fit(x_train)
    print pca.explained_variance_ratio_
    print sum(pca.explained_variance_ratio_)

可以看到，效果并不是很理想，前10个新特征总共表现了原来50%的变化，不过也比原来好了很多，先用这个看看。

    x_train_pca = pd.DataFrame(pca.transform(x_train))
    x_valid_pca = pd.DataFrame(pca.transform(x_valid))
    x_train_pca.info()

### 模型构建

导入库、训练数据处理

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_matrix = xgb.DMatrix(x_train, y_train)
valid_matrix = xgb.DMatrix(x_valid, y_valid)
watchlist = [(train_matrix, 'train'), (valid_matrix, 'valid')]

### 模型训练

#### 第一版模型训练

xgboost基本调参方式：
1. 选择一个较大的学习率，一般在0.05到0.3，此处我们直接选择0.3，然后设置一个该学习率对应下合适的n_estimators。
2. 调参max_depth、min_child_weight，这两个参数对结果影响很大。
3. 调参subsample、colsample_bytree。
4. 调参gamma。
5. 降低learning_rate，再匹配一个合适的n_estimators。

初版参数

In [ ]:
ps_first = {
    'max_depth':5,
    'learning_rate':.3,
    'n_estimators':5000,
    'objective':'reg:linear',
    'booster':'gbtree',
    'gamma':0,
    'min_child_weight':1,
    'subsample':1,
    'colsample_bytree':1,
    'random_state':6,
    'silent':True
}

params_first = {
    "objective": "reg:linear",
    "booster" : "gbtree",
    "eta": 0.1,
    "max_depth": 5,
    "silent": 1,
    "seed": 6}
num_boost_round_first = 1000

In [ ]:
def train(params, num_boost_round):
    print('XGBoost Model Train Start....')
    start_time = time.time()
    model = xgb.train(params, train_matrix, num_boost_round, evals=watchlist, early_stopping_rounds=100)
    print('XGBoost Model Train End, Time: {:4f} s....'.format(time.time()-start_time))
    return model

def train2(ps, x, y, x_test, y_test):
    print('XGBRegressor Train Start....')
    start_time = time.time()
    model = XGBRegressor(max_depth=ps['max_depth'], learning_rate=ps['learning_rate'], 
                         n_estimators=ps['n_estimators'],objective=ps['objective'], silent=ps['silent'],
                         booster=ps['booster'], gamma=ps['gamma'], min_child_weight=ps['min_child_weight'],
                         subsample=ps['subsample'], colsample_bytree=ps['colsample_bytree'],
                        random_state=ps['random_state'], n_jobs=-1)
    model.fit(x, y, early_stopping_rounds=100, eval_set=[(x_test,y_test)], verbose=True)
    print('XGBRegressor Train End, Time: {:4f} s....'.format(time.time()-start_time))
    return model

In [ ]:
#model_first = train(params_first, num_boost_round_first)
model_first = train2(ps_first, x_train, y_train, x_valid, y_valid)

#### 训练情况

1. ps_first = 
        {
        'max_depth':5,
        'learning_rate':.3,
        'n_estimators':5000,
        'objective':'reg:linear',
        'booster':'gbtree',
        'gamma':0,
        'min_child_weight':1,
        'subsample':1,
        'colsample_bytree':1,
        'random_state':6,
        'silent':True,
        }
        
结果：在1077次时达到最优迭代，RMSPE为0.160147，花费时间955s，且连续100次误差不能降低导致训练提前终止，因此对于学习0.3来说，n_estimators设置为1100是合适的。

#### 定义Cross Validation函数

In [ ]:
def cv(params, num_boost_round):
    print('XGBoost Model cv Start....')
    start_time = time.time()
    cv = xgb.cv(params, train_matrix, num_boost_round, early_stopping_rounds=100)
    print('XGBoost Model cv End, Time: {:4f} s....'.format(time.time()-start_time))
    return cv

### 模型预测

In [ ]:
def predict(model, x_valid, y_valid):
    print('XGBoost Model Valid Start....')
    start_time = time.time()
    pred_valid = model.predict(xgb.DMatrix(x_valid))
    rmspe_value = rmspe(np.expm1(pred_valid), np.expm1(y_valid))
    print('Valid RMSPE:'+str(rmspe_value))
    print('XGBoost Model Valid End, Time: {:4f} s....'.format(time.time()-start_time))
    return pred_valid, rmspe_value

def predict2(model, x_valid, y_valid):
    print('XGBoost Model Valid Start....')
    start_time = time.time()
    pred_valid = model.predict(x_valid)
    rmspe_value = rmspe(np.expm1(pred_valid), np.expm1(y_valid))
    print('Valid RMSPE:'+str(rmspe_value))
    print('XGBoost Model Valid End, Time: {:4f} s....'.format(time.time()-start_time))
    return pred_valid, rmspe_value
    
pred_valid_first, rmspe_first = predict2(model_first, x_valid, y_valid)

RMSPE值达到0.1646，花费时间4s，基本与训练时看到的数据相符，也就是略高一些。

### 模型保存

In [ ]:
model_first.save_model('./model/first.model')

model_first.save_model('./model/first.model')

### 模型优化

#### 参数优化 - 网格搜索最优参数

In [ ]:
ps_opt_estimators = {
    'max_depth':5,
    'learning_rate':.3,
    'n_estimators':1100,
    'objective':'reg:linear',
    'booster':'gbtree',
    'gamma':0,
    'min_child_weight':1,
    'subsample':1,
    'colsample_bytree':1,
    'random_state':6,
    'silent':True,
}

##### max_depth和min_child_weight最优参数组合

优先调试这两个的原因是它们对结果的影响比较大，方便我们更快的得到更好的结果。

* min_child_weight [default=1]
  
      定义了一个子集的所有观察值的最小权重和。 
      这个可以用来减少过拟合，但是过高的值也会导致欠拟合，因此可以通过CV来调整min_child_weight。

* max_depth [default=6]
  
      树的最大深度，值越大，树越复杂。 
      这个可以用来控制过拟合，典型值是3-10。

目前max_depth=5, min_child_weight=1

In [ ]:
param_grid_maxdepth_minchildweight = {
 'max_depth':[5, 7, 9],
 'min_child_weight':[1, 3, 5]
}

In [ ]:
def gridSearch(ps, param_grid, X, Y):
    print('XGBRegressor Grid Search Start....')
    start_time = time.time()
    model = XGBRegressor(max_depth=ps['max_depth'], learning_rate=ps['learning_rate'], 
                         n_estimators=ps['n_estimators'],objective=ps['objective'], silent=ps['silent'],
                         booster=ps['booster'], gamma=ps['gamma'], min_child_weight=ps['min_child_weight'],
                         subsample=ps['subsample'], colsample_bytree=ps['colsample_bytree'], random_state=ps['random_state'], n_jobs=-1)
    
    grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=3)
    grid_result = grid_search.fit(X, Y)
    
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    
    print('XGBRegressor Grid Search End, Time: {:4f} s....'.format(time.time()-start_time))
    return grid_result

grid_result = gridSearch(ps_opt_estimators, param_grid_maxdepth_minchildweight, x_train, y_train)

网格搜索花费时间s，最优组合为：
* max_depth:
* min_child_weight:

下面使用最优组合训练一个模型，并计算下在验证集上的RMSPE值。

ps_opt_estimators_maxdepth_minchildweight = {
    'max_depth':,
    'learning_rate':.3,
    'n_estimators':1100,
    'objective':'reg:linear',
    'booster':'gbtree',
    'gamma':0,
    'min_child_weight':,
    'subsample':1,
    'colsample_bytree':1,
    'random_state':6,
    'silent':True,
}

model_estimators_maxdepth_minchildweight = train2(ps_opt_estimators_maxdepth_minchildweight, x_train, y_train, x_valid, y_valid)
pred_valid_estimators_maxdepth_minchildweight, rmspe_estimators_maxdepth_minchildweight = predict2(model_estimators_maxdepth_minchildweight, x_valid, y_valid)

优化max_depth、min_child_weight的模型训练时间s，验证集上RMSPE值，能够看到较优化前的0.1646，还是

##### 模型保存

model_estimators_maxdepth_minchildweight.save_model('/home/kael/projects/model/estimators_maxdepth_minchildweight.model')

##### gamma


gamma [default=0, alias: min_split_loss]

      这个指定了一个结点被分割时，所需要的最小损失函数减小的大小。 
      这个值一般来说需要根据损失函数来调整。
  
目前gamma为0。

param_grid_gamma = {
 'gamma':[i/10.0 for i in range(0,5,2)]
}
grid_result2 = gridSearch(ps_opt_estimators_maxdepth_minchildweight, param_grid_gamma, x_train, y_train)

看到，最优的gamma值为。

ps_opt_estimators_maxdepth_minchildweight_gamma = {
    'max_depth':,
    'learning_rate':.3,
    'n_estimators':1100,
    'objective':'reg:linear',
    'booster':'gbtree',
    'gamma':,
    'min_child_weight':,
    'subsample':1,
    'colsample_bytree':1,
    'random_state':6,
    'silent':True,
}

model_estimators_maxdepth_minchildweight_gamma = train2(ps_opt_estimators_maxdepth_minchildweight_gamma, 
                                                  x_train, y_train, x_valid, y_valid)
pred_valid_estimators_maxdepth_minchildweight_gamma, rmspe_estimators_maxdepth_minchildweight_gamma = predict2(
    model_estimators_maxdepth_minchildweight_gamma, x_valid, y_valid)

优化gamma的模型训练时间s，验证集上RMSPE值，能够看到较优化前的，还是

##### 模型保存

model_estimators_maxdepth_minchildweight_gamma.save_model('/home/kael/projects/model/estimators_maxdepth_minchildweight_gamma.model')

##### subsample 和colsample_bytree

* subsample [default=1]

        样本的采样率，如果设置成0.5，那么Xgboost会随机选择一般的样本作为训练集。
* colsample_bytree [default=1]

        构造每棵树时，列采样率（一般是特征采样率）。
        
当前subsample为1，colsample_bytree为1。

param_grid_subsample_colsample_bytree = {
    'subsample':[.7, .8, .9],
    'colsample_bytree':[.7, .8, .9]
}
grid_result3 = gridSearch(ps_opt_estimators_maxdepth_minchildweight_gamma, 
                          param_grid_subsample_colsample_bytree, x_train, y_train)

看到，最优的组合为：
* subsample:
* colsample_bytree:

ps_opt_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree = {
    'max_depth':,
    'learning_rate':.3,
    'n_estimators':1100,
    'objective':'reg:linear',
    'booster':'gbtree',
    'gamma':,
    'min_child_weight':,
    'subsample':,
    'colsample_bytree':,
    'random_state':6,
    'silent':True,
}

model_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree = train2(ps_opt_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree, 
                                                  x_train, y_train, x_valid, y_valid)
pred_valid_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree, rmspe_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree = predict2(
    model_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree, x_valid, y_valid)

优化gamma的模型训练时间s，验证集上RMSPE值，能够看到较优化前的，还是

##### 模型保存

model_estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree.save_model('/home/kael/projects/model/estimators_maxdepth_minchildweight_gamma_subsample_colsamplebytree.model')

##### 减小学习率，重新设置n_estimators

##### 优化后的参数

In [ ]:
ps_opt = {
    'max_depth':9,
    'learning_rate':.03,
    'n_estimators':6000,
    'objective':'reg:linear',
    'booster':'gbtree',
    'subsample':.9,
    'colsample_bytree':.7,
    'random_state':6,
    'silent':True,
}

params_opt = {
    "objective": "reg:linear",
    "booster" : "gbtree",
    "eta": 0.03,
    "max_depth": 10,
    'subsample':.9,
    'colsample_bytree':.7,
    "silent": 1,
    "seed": 6}

model_opt = train2(ps_opt,  x_train, y_train, x_valid, y_valid)
pred_valid_opt, rmspe_opt = predict2(model_opt, x_valid, y_valid)

#model_opt = train(params_opt, 6000)

可以看到优化后的RMSPE值从0.1646减低到了。

##### 模型保存

In [ ]:
model_opt.save_model('/home/kael/projects/model/opt.model

#### 校正系数：校正整体偏差

观察下预测值和真值的分布情况。

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(range(len(np.expm1(pred_valid)[::1115])), np.expm1(pred_valid)[::1115], color='red')
plt.scatter(range(len(np.expm1(y_valid)[::1115])), np.expm1(y_valid)[::1115], color='blue')

In [ ]:
np.mean(np.abs(np.expm1(pred_valid)-np.expm1(y_valid)))

In [ ]:
np.mean(np.expm1(pred_valid)-np.expm1(y_valid))

我们的数据整体上是低于真实值的，因此使用校正系数来整体校正偏差是合理的。

In [ ]:
def get_fix_actor(pred_valid, y_valid):
    results = {}
    for actor in [0.990+i/1000. for i in range(20)]:
        results[actor]=rmspe(y_pred=np.expm1(pred_valid)*actor, y_real=np.expm1(y_valid))
    return sorted(results.items(),key = lambda x:x[1],reverse = True)[-1]

print '校正前：'+str(rmspe(np.expm1(pred_valid), np.expm1(y_valid)))
print '校正后：'
actor_score = get_fix_actor(pred_valid, y_valid)
print actor_score

恩恩，效果还不错，下降了0.01，最佳校正系数为0.99，看看加上校正系数后的数据与校正前、真实数据对比情况：

In [ ]:
plt.figure(figsize=(15,15))
plt.scatter(range(len(np.expm1(pred_valid)[::1115])), np.expm1(pred_valid)[::1115], color='red')
plt.scatter(range(len(np.expm1(pred_valid)[::1115])), np.expm1(pred_valid*actor_score[0])[::1115], color='blue')
plt.scatter(range(len(np.expm1(y_valid)[::1115])), np.expm1(y_valid)[::1115], color='green')

#### 多模型融合

In [ ]:
num_model = 5

print 'XGBoost XModel Train Start....'
start_time = time.time()
models = []
for i in range(num_model):
    params = {
        'objective':'reg:linear', 
        'booster':'gbtree', # 注意此处的提升方式，保证了我们之前一些类别字段映射到01234也是可行的，而不需增加维度
        'eta':.03,
        'max_depth':10,
        'subsample':.9,
        'colsample_bytree':.7,
        'silent':1,
        'seed':10000+i
    }
    num_boost_round = 5000
    model = xgb.train(params, train_matrix, num_boost_round, evals=watchlist, 
                  early_stopping_rounds=100)
    models.append(model)

print 'XGBoost XModel Train End, Time: {:4f} s....'.format(time.time()-start_time)

多模型训练中我们发现，当前参数下，模型训练多次在4500+次迭代时达到最优，5个模型的总耗时为：26583s，也就是7.4个小时这样。。。一晚上。。。

#### 多模型校正系数

In [ ]:
actor_scores = []
for i in range(len(models)):
    pred_valid = models[i].predict(xgb.DMatrix(x_valid))
    actor_scores.append(get_fix_actor(pred_valid, y_valid))

In [ ]:
actor_scores

In [ ]:
weights = []
for i in range(len(actor_scores)):
    weights.append(actor_scores[i][1])
weights = [sum(weights)-w for w in weights]
weights = [1.*w/sum(weights) for w in weights]

In [ ]:
weights

#### 定义多模型预测函数

In [ ]:
def predict_x(x_valid):
    preds = []
    for m in models:
        preds.append(m.predict(xgb.DMatrix(x_valid)))
    for i in range(len(preds)):
        preds[i] = [p*actor_scores[i][0]*weights[i]for p in preds[i]]
    final_pred = []
    for i in range(len(preds[0])):
        p=0
        for j in range(len(preds)):
            p+=preds[j][i]
        final_pred.append(p)
    return final_pred

print 'X模型融合RMSPE:'+str(rmspe(np.expm1(predict_x(x_valid)), np.expm1(y_valid)))

#### 保存多模型

In [ ]:
for i in range(len(models)):
    models[i].save_model('/home/kael/projects/model/model_'+str(i)+'.model')

### 单模型、多模型融合、真实值对比

#### 数据可视化

In [ ]:
plt.figure(figsize=(15,15))
singal_model_pred_valid = model.predict(xgb.DMatrix(x_valid))*actor_score[0]
x_model_pred_valid = predict_x(x_valid)
plt.scatter(range(len(y_valid[:100])), np.expm1(singal_model_pred_valid[:100]), color='red')
plt.scatter(range(len(y_valid[:100])), np.expm1(x_model_pred_valid[:100]), color='blue')
plt.scatter(range(len(y_valid[:100])), np.expm1(y_valid[:100]), color='green')

#### 验证集上RMSPE对比

## kaggle上对比

In [ ]:
test_id = test_all.Id
test_all.drop(['Id'], axis=1, inplace=True)

### kaggle上对比基准模型、单模型、多模型的表现

#### 基准模型测试

In [ ]:
pd.DataFrame({'Id':test_id, 'Sales':pd.Series([pred_base]*len(test_id))}).to_csv('submission_base.csv', index=False)

#### kaggle反馈

得分情况：
* private score:0.17406
* publice score:0.16244

![基准模型截图](image/kaggle_base.png)

#### 单模型测试

In [ ]:
pred_test = model.predict(xgb.DMatrix(test_all))
pred_test = pred_test*actor_score[0] # 校正系数

In [ ]:
pd.DataFrame({'Id':test_id, 'Sales':np.expm1(pred_test)}).to_csv('submission.csv', index=False)

#### kaggle反馈

得分情况：
* private score:0.17406
* publice score:0.16244

![单模型截图](image/kaggle.png)

### 多模型融合测试

In [ ]:
pred_x_test = predict_x(test_all)

In [ ]:
pd.DataFrame({'Id':test_id, 'Sales':np.expm1(pred_x_test)}).to_csv('submission_x.csv', index=False)

多模型融合的得分情况：
* private score:
* publice score:

![多模型得分截图](image/kaggle_xmodel.png)

### 增量训练验证集-各个model，包括参数优化后的单模型和多模型

#### 各模型增量训练验证集

#### 单模型、多模型在增量训练后在kaggle的表现

## 结论

到底是多模型融合呢，还是单模型呢。。。。